In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
data_path = './3rd-ml100marathon-midterm/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')
print(df_train.shape)
print(df_test.shape)

(113, 22)
(33, 21)


In [3]:
# 重組資料成為訓練 / 預測用格式

# poi 這個 column 是只有 train set 才有的 (label)
train_Y = df_train['poi']
df_train = df_train.drop(['poi'] , axis=1) #將 column = 'poi' drop 出來
train_num = df_train.shape[0]

df = pd.concat([df_train, df_test])
print(df.shape)
df.head(10)

(146, 21)


,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0
5,HIRKO JOSEPH,NaN,10259.0,NaN,NaN,joe.hirko@enron.com,30766064.0,77978.0,NaN,NaN,...,NaN,NaN,2856.0,NaN,NaN,NaN,NaN,NaN,91093.0,30766064.0
6,YEAGER F SCOTT,NaN,NaN,NaN,NaN,scott.yeager@enron.com,8308552.0,53947.0,NaN,NaN,...,NaN,NaN,147950.0,3576206.0,NaN,158403.0,NaN,NaN,360300.0,11884758.0
7,COLWELL WESLEY,1200000.0,27610.0,-144062.0,NaN,wes.colwell@enron.com,NaN,16514.0,40.0,240.0,...,NaN,NaN,101740.0,698242.0,NaN,288542.0,1132.0,1758.0,1490344.0,698242.0
8,LAY KENNETH L,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,...,81525000.0,3600000.0,10359729.0,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,GLISAN JR BEN F,600000.0,NaN,NaN,NaN,ben.glisan@enron.com,384728.0,125978.0,16.0,52.0,...,NaN,71023.0,200308.0,393818.0,NaN,274975.0,874.0,873.0,1272284.0,778546.0


In [4]:
print(df.columns)
ratio = 0.4
selected_columns = [] #只留下 非 NaN 值比率大於 ratio 的 columns 
for col in df.columns:
    if (((~df_train[col].isnull()).values.sum())/df.shape[0] > ratio):
        selected_columns.append(col)
print(selected_columns)
df = df[selected_columns]

Index(['name', 'bonus', 'deferral_payments', 'deferred_income',
       'director_fees', 'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
       'loan_advances', 'long_term_incentive', 'other', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
       'to_messages', 'total_payments', 'total_stock_value'],
      dtype='object')
['name', 'bonus', 'email_address', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'other', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']


In [5]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

0 Integer Features : []

13 Float Features : ['bonus', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'other', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']

2 Object Features : ['name', 'email_address']


In [6]:
#Category Encoding
#name 是key 所以不處理
#對 email_address 做 LabelEncoder

df['email_address'] = df['email_address'].fillna('None')
df['email_address'] = LabelEncoder().fit_transform(df['email_address'])

In [7]:
df.head(10)

,name,bonus,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,56,19794175.0,46950.0,18.0,42.0,4.0,174839.0,2748364.0,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,36,19250000.0,29336.0,108.0,88.0,30.0,22122.0,6843672.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,83,1624396.0,22884.0,39.0,13.0,14.0,1573324.0,869220.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,73,NaN,118134.0,NaN,NaN,NaN,907502.0,985032.0,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,10,NaN,35818.0,144.0,199.0,25.0,486.0,126027.0,240189.0,2188.0,2598.0,1639297.0,126027.0
5,HIRKO JOSEPH,NaN,45,30766064.0,77978.0,NaN,NaN,NaN,2856.0,NaN,NaN,NaN,NaN,91093.0,30766064.0
6,YEAGER F SCOTT,NaN,98,8308552.0,53947.0,NaN,NaN,NaN,147950.0,3576206.0,158403.0,NaN,NaN,360300.0,11884758.0
7,COLWELL WESLEY,1200000.0,111,NaN,16514.0,40.0,240.0,11.0,101740.0,698242.0,288542.0,1132.0,1758.0,1490344.0,698242.0
8,LAY KENNETH L,7000000.0,57,34348384.0,99832.0,36.0,123.0,16.0,10359729.0,14761694.0,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,GLISAN JR BEN F,600000.0,4,384728.0,125978.0,16.0,52.0,6.0,200308.0,393818.0,274975.0,874.0,873.0,1272284.0,778546.0


In [8]:
df['bonus']

0     1750000.0
1     5600000.0
2      200000.0
3      800000.0
4     1250000.0
        ...    
28    1000000.0
29    1500000.0
30          NaN
31     200000.0
32    2600000.0
Name: bonus, Length: 146, dtype: float64

In [9]:
df['bonus'][(~df['bonus'].isnull())]
#畫出每個 Float Feature 的分布
# for f in float_features:
#     df[f][(~df[col].isnull())].hist(bins = 30)
#     plt.xlabel(f)
#     plt.ylabel('Counts')
# plt.show() 

0     1750000.0
1     5600000.0
2      200000.0
3      800000.0
4     1250000.0
        ...    
25     300000.0
28    1000000.0
29    1500000.0
31     200000.0
32    2600000.0
Name: bonus, Length: 82, dtype: float64

In [10]:
#Numeric encoding


In [11]:
df_sample_submission = pd.read_csv(data_path + 'sample_submission.csv')
print(df_sample_submission)

                    name       poi
0       BELDEN TIMOTHY N  0.007201
1     BOWEN JR RAYMOND M  0.903340
2         HANNON KEVIN P  0.572731
3       DELAINEY DAVID W  0.950927
4       CAUSEY RICHARD A  0.942402
5       HICKERSON GARY J  0.013530
6         FREVERT MARK A  0.275866
7            CHAN RONNIE  0.144731
8   DONAHUE JR JEFFREY M  0.388124
9      REYNOLDS LAWRENCE  0.032208
10      HORTON STANLEY C  0.300574
11         LEWIS RICHARD  0.616451
12    SHERRICK JEFFREY B  0.440055
13        MEYER JEROME J  0.657208
14   DERRICK JR. JAMES V  0.833014
15              PIRO JIM  0.413224
16   DETMERING TIMOTHY J  0.720923
17     KISHKILL JOSEPH G  0.904454
18        BAY FRANKLIN R  0.298674
19        PRENTICE JAMES  0.148842
20                 TOTAL  0.351439
21        FALLON JAMES B  0.284129
22            METTS MARK  0.568772
23         WODRASKA JOHN  0.890761
24   MORDAUNT KRISTINA M  0.838412
25      CARTER REBECCA C  0.354295
26      BHATNAGAR SANJAY  0.581497
27      WALTERS GARE